In [1]:
!python -V

Python 3.9.25


In [2]:
import pandas as pd
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [5]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("my-nyc-taxi-experiment")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1768319851381, experiment_id='2', last_update_time=1768319851381, lifecycle_stage='active', name='my-nyc-taxi-experiment', tags={}>

In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    pd.to_datetime(df.lpep_dropoff_datetime)
    pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [8]:
len(df_train), len(df_val)

(73908, 61921)

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [10]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse = mean_squared_error(y_val, y_pred) ** 0.5
print(rmse)

7.758715208009878


In [13]:
with open('models/lin_reg.bin', 'wb') as f_out: 
    pickle.dump((dv, lr), f_out)

In [14]:
with mlflow.start_run():

    mlflow.set_tag("developer", "ksenia")

    mlflow.log_param("train-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
    
    alpha = 0.001
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred) ** 0.5
    print(rmse)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

9.233436225720547
🏃 View run magnificent-wren-377 at: http://127.0.0.1:5000/#/experiments/2/runs/3e816a3fe742412d96d840923ec5080e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


In [15]:
lr = Ridge()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse = mean_squared_error(y_val, y_pred) ** 0.5
print(rmse)

7.70373515548786


In [16]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [17]:
train = xgb.DMatrix(X_train, label = y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [18]:
def objective(params):
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred) ** 0.5
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [ ]:
params = {
    'learning_rate' : 0.1240313820257843,
    'max_depth' : 44,
    'min_child_weight' : 2.1422561722534965,
    'objective' : 'reg:linear',
    'reg_alpha' : 0.04312386269914251,
    'reg_lambda' : 0.0326805234852081,
    'seed' : 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=100,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

In [20]:
mlflow.xgboost.autolog(disable=True)

In [21]:
with mlflow.start_run():

    train = xgb.DMatrix(X_train, label = y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    
    best_params = {
    'learning_rate' : 0.1240313820257843,
    'max_depth' : 44,
    'min_child_weight' : 2.1422561722534965,
    'objective' : 'reg:linear',
    'reg_alpha' : 0.04312386269914251,
    'reg_lambda' : 0.0326805234852081,
    'seed' : 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred) ** 0.5
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
    

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:16:55] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.21388
[1]	validation-rmse:10.37564
[2]	validation-rmse:9.67585
[3]	validation-rmse:9.09538
[4]	validation-rmse:8.61768
[5]	validation-rmse:8.22493
[6]	validation-rmse:7.90586
[7]	validation-rmse:7.64610
[8]	validation-rmse:7.43767
[9]	validation-rmse:7.26686
[10]	validation-rmse:7.12970
[11]	validation-rmse:7.01709
[12]	validation-rmse:6.92560
[13]	validation-rmse:6.85209
[14]	validation-rmse:6.79218
[15]	validation-rmse:6.74323
[16]	validation-rmse:6.70147
[17]	validation-rmse:6.66779
[18]	validation-rmse:6.63984
[19]	validation-rmse:6.61572
[20]	validation-rmse:6.59639
[21]	validation-rmse:6.57960
[22]	validation-rmse:6.56555
[23]	validation-rmse:6.55310
[24]	validation-rmse:6.54215
[25]	validation-rmse:6.53275
[26]	validation-rmse:6.52511
[27]	validation-rmse:6.51824
[28]	validation-rmse:6.51180
[29]	validation-rmse:6.50606
[30]	validation-rmse:6.50118
[31]	validation-rmse:6.49609
[32]	validation-rmse:6.49150
[33]	validation-rmse:6.48833
[34]	validation-rmse:6

2026/01/14 09:17:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:17:47] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2026/01/14 09:17:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run zealous-elk-633 at: http://127.0.0.1:5000/#/experiments/2/runs/9626268a89af405fb4fd7525d8c67107
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


In [29]:
model_uri = "mlflow-artifacts:/2/models/m-e4370d674de5463a8fcd4ebc9e3822f9/artifacts"

loaded_model = mlflow.pyfunc.load_model(model_uri)

In [30]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: mlflow-artifacts:/2/models/m-e4370d674de5463a8fcd4ebc9e3822f9/artifacts
  flavor: mlflow.xgboost
  run_id: 9626268a89af405fb4fd7525d8c67107

In [31]:
xgboost_model = mlflow.xgboost.load_model(model_uri)

In [32]:
xgboost_model

In [33]:
y_pred = xgboost_model.predict(valid)

In [34]:
y_pred[:10]

array([14.928554 ,  7.1066146, 16.821402 , 24.426222 ,  9.814736 ,
       17.117676 , 13.140835 ,  8.588156 ,  9.054085 , 20.201723 ],
      dtype=float32)